In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/alx-movie-recommendation-project-2024/sample_submission.csv
/kaggle/input/alx-movie-recommendation-project-2024/movies.csv
/kaggle/input/alx-movie-recommendation-project-2024/imdb_data.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_scores.csv
/kaggle/input/alx-movie-recommendation-project-2024/train.csv
/kaggle/input/alx-movie-recommendation-project-2024/test.csv
/kaggle/input/alx-movie-recommendation-project-2024/tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/links.csv


In [2]:
from tqdm import tqdm
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise import Reader, Dataset, SVDpp, KNNWithMeans, SlopeOne
from surprise.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Install packages here
# Packages for data processing
import numpy as np
import pandas as pd
from tqdm import tqdm
import heapq
import datetime
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from scipy.sparse import csr_matrix
import scipy as sp


# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import heapq

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

from tqdm import tqdm
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise import Reader, Dataset, SVDpp, KNNWithMeans, SlopeOne
from surprise.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd

# Load the data
genome_scores = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/genome_scores.csv')
genome_tags = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/genome_tags.csv')
imdb_data = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/imdb_data.csv')
links = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/links.csv')
tags = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/tags.csv')
train = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/train.csv')
movies = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/movies.csv')
test = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/test.csv')

# Display the first few rows of each DataFrame
print(genome_scores.head())
print(genome_tags.head())
print(imdb_data.head())
print(links.head())
print(tags.head())
print(train.head())
print(movies.head())
print(test.head())


   movieId  tagId  relevance
0        1      1    0.02875
1        1      2    0.02375
2        1      3    0.06250
3        1      4    0.07575
4        1      5    0.14075
   tagId           tag
0      1           007
1      2  007 (series)
2      3  18th century
3      4         1920s
4      5         1930s
   movieId                                         title_cast  \
0        1  Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...   
1        2  Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...   
2        3  Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...   
3        4  Whitney Houston|Angela Bassett|Loretta Devine|...   
4        5  Steve Martin|Diane Keaton|Martin Short|Kimberl...   

              director  runtime       budget  \
0        John Lasseter     81.0  $30,000,000   
1   Jonathan Hensleigh    104.0  $65,000,000   
2  Mark Steven Johnson    101.0  $25,000,000   
3       Terry McMillan    124.0  $16,000,000   
4       Albert Hackett    106.0  $30,000,000   

        

In [5]:
print("Train: ")
print(str(train.isnull().sum()))
print("************")
print("Test: ")
print(str(test.isnull().sum()))
print("************")
print("Movies: ")
print(str(movies.isnull().sum()))
print("************")
print("Links: ")
print(str(links.isnull().sum()))
print("************")
print("IMDB: ")
print(str(imdb_data.isnull().sum()))
print("************")
print("Genome scores: ")
print(str(genome_scores.isnull().sum()))
print("************")
print("Genome tags: ")
print(str(genome_tags.isnull().sum()))

Train: 
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
************
Test: 
userId     0
movieId    0
dtype: int64
************
Movies: 
movieId    0
title      0
genres     0
dtype: int64
************
Links: 
movieId      0
imdbId       0
tmdbId     107
dtype: int64
************
IMDB: 
movieId              0
title_cast       10068
director          9874
runtime          12089
budget           19372
plot_keywords    11078
dtype: int64
************
Genome scores: 
movieId      0
tagId        0
relevance    0
dtype: int64
************
Genome tags: 
tagId    0
tag      0
dtype: int64


In [6]:
# Prepare data for Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

In [7]:
svd = SVD()
trainset = data.build_full_trainset()
svd.fit(trainset)

In [8]:
# Create a set of test pairs
test_pairs = set(test.apply(lambda row: f"{row['userId']}_{row['movieId']}", axis=1))

def get_predictions_batch(svd, test_pairs, batch_size=1000):
    predictions = {}
    for i in tqdm(range(0, len(test_pairs), batch_size)):
        batch_pairs = list(test_pairs)[i:i+batch_size]
        for pair in batch_pairs:
            user, movie = map(int, pair.split('_'))
            est = svd.predict(user, movie).est
            predictions[pair] = est
    return predictions

In [9]:
print("Getting predictions...")
predictions = get_predictions_batch(svd, test_pairs)

Getting predictions...


100%|██████████| 5001/5001 [29:00<00:00,  2.87it/s]


In [10]:
print("Creating submission DataFrame...")
submission = pd.DataFrame.from_dict(predictions, orient='index', columns=['rating'])
submission.index.name = 'Id'
submission.reset_index(inplace=True)

Creating submission DataFrame...


In [11]:
print("Saving submission to CSV...")
submission.to_csv('submission.csv', index=False)
print("Done!")

Saving submission to CSV...
Done!


In [12]:
submission.head()

,Id,rating
0,147059_592,3.828160
1,118124_2406,3.957665
2,71919_778,4.156155
3,134985_88405,3.463283
4,4741_2706,4.202980
